In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# TPU settings

In [ ]:
import math, re, os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from skimage import io
import matplotlib.pyplot as plt

# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:


print("Tensorflow version " + tf.__version__)


# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)


In [ ]:
import pandas as pd

train = pd.read_csv('../input/prostate-cancer-grade-assessment/train.csv')

train.head()

In [ ]:
from skimage import io
import matplotlib.pyplot as plt

filrt = "../input/prostate-cancer-grade-assessment/train_images/"

# print("../input/prostate-cancer-grade-assessment/train_images/"+train["image_id"][0])


# for x,i in zip(train["image_id"],range(10)):
    
# #     read the image stack
#     img = io.imread(filrt+x+".tiff")
#     # show the image
#     plt.imshow(img,cmap='gray')
#     plt.axis('off')
#     plt.show()


In [ ]:
for x,i in zip(filrt+train["image_id"]+".tiff",range(10)):
    print(x.shape)

In [ ]:
img = io.imread(filrt+train["image_id"][0]+".tiff")
img.resize(int(29440/10),int(27648/10),3)

plt.imshow(img,cmap='gray')
plt.axis('off')
plt.show()
# img/255.0

In [ ]:
tfio.experimental.image.decode_tiff(
    train["image_id"], index=0, name=None
)

In [ ]:
EPOCHS = 5
with strategy.scope():
    pretrained_model = tf.keras.applications.DenseNet201(
        weights='imagenet',
        include_top=False ,
        input_shape=img.shape
    )
    pretrained_model.trainable = False
    
    model = tf.keras.Sequential([
        # To a base pretrained on ImageNet to extract features from images...
        pretrained_model,
        # ... attach a new head to act as a classifier.
#         tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(5, activation='softmax')
    ])
    
model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)
wide = model 

model.summary()

# Define training epochs
EPOCHS = 5
STEPS_PER_EPOCH = 12

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

history = model.fit(
    filrt+train["image_id"]+".tiff",train["isup_grade"],
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[early_stopping],
)

In [ ]:
train["isup_grade"]

In [ ]:
[img.shape]

In [ ]:
import pathlib
dataset_url = "../input/prostate-cancer-grade-assessment/train_images"
# data_dir = tf.keras.utils.get_file(origin=dataset_url, 
#                                    fname='flower_photos', 
#                                    untar=True)
data_dir = pathlib.Path(dataset_url)

In [ ]:
image_count = len(list(data_dir.glob('*.tiff')))
print(image_count)
imm = None

for x in data_dir.glob('*.tiff'):
#     os.rename(x, "dst"+str(x))
    print(x)
    imm = x
    break

In [ ]:
data_dir

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#Generate a dataset

image_size = (28, 28)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

In [ ]:
!pip install fastai2
!pip install fastai

In [ ]:
from fastai2  import *


In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *

In [ ]:
# CLICK ME
from fastai.vision.all import *
path = untar_data(URLs.PETS)/'images'

def is_cat(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

In [ ]:

path = untar_data(URLs.CAMVID_TINY)
dls = SegmentationDataLoaders.from_label_func(
    path, bs=8, fnames = get_image_files(path/"images"),
    label_func = lambda o: path/'labels'/f'{o.stem}_P{o.suffix}',
    codes = np.loadtxt(path/'codes.txt', dtype=str)
)

learn = unet_learner(dls, resnet34)
learn.fine_tune(8)

In [ ]:
learn.show_results(max_n=6, figsize=(7,8))


from fastai.text.all import *

dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.fine_tune(4, 1e-2)

In [ ]:
learn.predict("I really liked that movie!")